In [1]:
import pandas as pd

In [30]:
app_att = pd.read_csv("../data_source/apartment_attributes.csv")
user_viewings = pd.read_csv("../data_source/user_viewings.csv")
app = pd.read_csv("../data_source/apartment.csv")
booking = pd.read_csv("../data_source/booking.csv")

In [17]:
app.head()

,id,title,source,price,currency,listing_created_on,is_active,last_modified_timestamp
0,1,"Johnson, Fowler and Johnson",Realtor,4020.95,EUR,27/03/2024,True,27/12/2021
1,2,Kelley-Johnson,Craigslist,1390.01,INR,23/07/2023,True,10/02/2023
2,3,"Gibson, Delgado and Austin",Craigslist,3082.67,INR,15/04/2024,True,02/01/2020
3,4,"Kelly, Smith and Gibson",Zillow,4724.20,USD,27/10/2020,False,23/06/2023
4,5,Carroll-Burch,Airbnb,3867.11,USD,01/02/2021,False,25/07/2022


In [8]:
booking['currency'] = 'USD'

In [14]:
# turn the date columns into datetime objects
user_viewings['viewed_at'] = pd.to_datetime(user_viewings['viewed_at'])

C:\Users\HeskeyAmoakoFordjour\AppData\Local\Temp\ipykernel_20280\560772600.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  user_viewings['viewed_at'] = pd.to_datetime(user_viewings['viewed_at'])


In [20]:
# Convert app date columns to datetime objects
app['listing_created_on'] = pd.to_datetime(app['listing_created_on'])
app['last_modified_timestamp'] = pd.to_datetime(app['last_modified_timestamp'])

C:\Users\HeskeyAmoakoFordjour\AppData\Local\Temp\ipykernel_20280\2185152953.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  app['last_modified_timestamp'] = pd.to_datetime(app['last_modified_timestamp'])


In [21]:
# Write the app data to a new csv file
app.to_csv("../data_source/apartment.csv", index=False)

In [16]:
# Write the data to csv file
user_viewings.to_csv('../data_source/user_viewings.csv', index=False)

In [31]:
booking.head()

,booking_id,user_id,apartment_id,booking_date,checkin_date,checkout_date,total_price,currency,booking_status
0,1,2920,130940,11/11/2022,07/09/2024,30/06/2023,2923.67,USD,confirmed
1,2,2788,191497,17/05/2024,07/07/2024,06/09/2021,4645.29,USD,pending
2,3,9285,139117,26/11/2022,12/02/2025,07/04/2023,1683.99,USD,canceled
3,4,8161,118131,09/07/2020,25/10/2024,08/02/2021,1870.62,USD,confirmed
4,5,6733,43470,08/05/2024,31/01/2022,29/10/2024,3379.44,USD,pending


In [37]:
booking.head(10)

,booking_id,user_id,apartment_id,booking_date,checkin_date,checkout_date,total_price,currency,booking_status,days_spent
0,1,2920,130940,2022-11-11,2024-09-07,2023-06-30,2923.67,USD,confirmed,-435 days
1,2,2788,191497,2024-05-17,2024-07-07,2021-09-06,4645.29,USD,pending,-1035 days
2,3,9285,139117,2022-11-26,2025-02-12,2023-04-07,1683.99,USD,canceled,-677 days
3,4,8161,118131,2020-07-09,2024-10-25,2021-02-08,1870.62,USD,confirmed,-1355 days
4,5,6733,43470,2024-05-08,2022-01-31,2024-10-29,3379.44,USD,pending,1002 days
5,6,8017,183666,2021-06-08,2020-08-29,2024-11-12,1596.56,USD,pending,1536 days
6,7,2051,97644,2022-09-08,2024-03-19,2025-02-27,3225.36,USD,pending,345 days
7,8,3889,17073,2025-01-24,2021-11-17,2021-08-03,3017.16,USD,confirmed,-106 days
8,9,8306,42199,2021-10-10,2023-07-10,2020-12-18,4867.80,USD,canceled,-934 days
9,10,8952,122566,2024-03-27,2023-07-01,2021-09-27,2216.02,USD,canceled,-642 days


In [32]:
# Write the date in the booking to datetime objects
booking['booking_date'] = pd.to_datetime(booking['booking_date'], dayfirst=True)
booking['checkin_date'] = pd.to_datetime(booking['checkin_date'], dayfirst=True)
booking['checkout_date'] = pd.to_datetime(booking['checkout_date'], dayfirst=True)

In [35]:
booking['days_spent'] = booking['checkout_date'] - booking['checkin_date']

In [34]:
# Turn the booking to a csv file
booking.to_csv("../data_source/booking.csv", index=False)

In [25]:
booking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   booking_id      500000 non-null  int64         
 1   user_id         500000 non-null  int64         
 2   apartment_id    500000 non-null  int64         
 3   booking_date    500000 non-null  datetime64[ns]
 4   checkin_date    500000 non-null  datetime64[ns]
 5   checkout_date   500000 non-null  datetime64[ns]
 6   total_price     500000 non-null  float64       
 7   currency        500000 non-null  object        
 8   booking_status  500000 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(3), object(2)
memory usage: 34.3+ MB


In [4]:
# Merge the dataframes
apartments = app_att.merge(app, on="id")

In [23]:
apartments.columns

Index(['id', 'category', 'body', 'amenities', 'bathrooms', 'bedrooms', 'fee',
       'has_photo', 'pets_allowed', 'price_display', 'price_type',
       'square_feet', 'address', 'cityname', 'state', 'latitude', 'longitude',
       'title', 'source', 'price', 'currency', 'listing_created_on',
       'is_active', 'last_modified_timestamp'],
      dtype='object')

In [24]:
user_viewings.columns

Index(['user_id', 'apartment_id', 'viewed_at', 'is_wishlisted',
       'call_to_action'],
      dtype='object')

In [7]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       10000 non-null  int64  
 1   category                 10000 non-null  object 
 2   body                     10000 non-null  object 
 3   amenities                10000 non-null  object 
 4   bathrooms                10000 non-null  float64
 5   bedrooms                 10000 non-null  float64
 6   fee                      0 non-null      float64
 7   has_photo                10000 non-null  object 
 8   pets_allowed             5837 non-null   object 
 9   price_display            10000 non-null  object 
 10  price_type               10000 non-null  object 
 11  square_feet              10000 non-null  int64  
 12  address                  10000 non-null  object 
 13  cityname                 9923 non-null   object 
 14  state                  

In [49]:
apartments.head()

,id,category,body,amenities,bathrooms,bedrooms,fee,has_photo,pets_allowed,price_display,...,state,latitude,longitude,title,source,price,currency,listing_created_on,is_active,last_modified_timestamp
0,5668626895,housing/rent/apartment,"This unit is located at second St NE, Uhland T...","Refrigerator,Stove,Air Conditioning,Laundry",1.0,0.0,NaN,Thumbnail,NaN,$790,...,DC,38.9057,-76.9861,"Studio apartment 2nd St NE, Uhland Terrace NE,...",RentLingo,790,USD,2024-05-16 06:56:13,1,2024-05-16 06:56:13
1,5664597177,housing/rent/apartment,"This unit is located at 814 Schutte Road, Evan...","Refrigerator,Stove,Air Conditioning,Laundry",1.0,1.0,NaN,Thumbnail,NaN,$425,...,IN,37.9680,-87.6621,Studio apartment 814 Schutte Road,RentLingo,425,USD,2024-05-16 06:35:35,1,2024-05-16 06:35:35
2,5668626833,housing/rent/apartment,"This unit is located at N Scott St, 14th St N,...","Refrigerator,Stove,Air Conditioning,Laundry",1.0,0.0,NaN,Thumbnail,NaN,"$1,390",...,VA,38.8910,-77.0816,"Studio apartment N Scott St, 14th St N, Arling...",RentLingo,1390,USD,2024-05-18 22:58:20,1,2024-05-21 02:47:32
3,5659918074,housing/rent/apartment,"This unit is located at 1717 12th Ave, Seattle...","Refrigerator,Stove,Air Conditioning,Laundry",1.0,0.0,NaN,Thumbnail,NaN,$925,...,WA,47.6160,-122.3275,Studio apartment 1717 12th Ave,RentLingo,925,USD,2024-05-17 16:18:36,0,2024-05-17 16:18:36
4,5668626759,housing/rent/apartment,"This unit is located at Washington Blvd, N Cle...","Refrigerator,Stove,Air Conditioning,Laundry",1.0,0.0,NaN,Thumbnail,NaN,$880,...,VA,38.8738,-77.1055,"Studio apartment Washington Blvd, N Cleveland ...",RentLingo,880,USD,2024-05-17 08:47:56,0,2024-05-19 02:36:48


In [50]:
apartments.columns

Index(['id', 'category', 'body', 'amenities', 'bathrooms', 'bedrooms', 'fee',
       'has_photo', 'pets_allowed', 'price_display', 'price_type',
       'square_feet', 'address', 'cityname', 'state', 'latitude', 'longitude',
       'title', 'source', 'price', 'currency', 'listing_created_on',
       'is_active', 'last_modified_timestamp'],
      dtype='object')

In [48]:
user_viewings.head()

,user_id,apartment_id,viewed_at,is_wishlisted,call_to_action
0,2947,5668632955,2024-05-19 22:53:46,y,contact_agent
1,338,5508874895,2024-05-22 05:16:28,y,shortlisted
2,4940,5668616269,2024-05-24 13:03:52,n,reported
3,2402,5509147865,2024-05-28 05:09:07,y,reported
4,3005,5661938296,2024-05-25 00:36:16,y,shortlisted


In [46]:
user_viewings['call_to_action'].value_counts()

call_to_action
reported         1702
contact_agent    1685
shortlisted      1612
Name: count, dtype: int64

In [8]:
apartments.isnull().sum()

id                             0
category                       0
body                           0
amenities                      0
bathrooms                      0
bedrooms                       0
fee                        10000
has_photo                      0
pets_allowed                4163
price_display                  0
price_type                     0
square_feet                    0
address                        0
cityname                      77
state                         77
latitude                      10
longitude                     10
title                          0
source                         0
price                          0
currency                       0
listing_created_on             0
is_active                      0
last_modified_timestamp        0
dtype: int64